In [65]:
#import libraries
import numpy as np
import cv2
import Image_Processing_Classes as ip
from matplotlib import pyplot as plt
import mediapipe as mp
import math
import os
import imutils
from IPython.display import Image
import pandas as pd
import re
from contextlib import contextmanager
import traceback

In [110]:

class BST_Image_Proc:
    #this class will serve as a way to incorporate multiple image processing libraries
    def __init__(self, raw_frame=None, vid=None, codec=None, raw_writer=None, processed_writer=None):    
        if vid==None:
            self.vid = cv2.VideoCapture(0)
        else:
            self.vid = vid
            
        if codec==None:
            self.codec = cv2.VideoWriter_fourcc(*'XVID')
        else:
            self.codec = codec
            
        if raw_writer==None:
            self.raw_writer = cv2.VideoWriter('output.avi', codec, 20.0, (640, 480))
        else:
            self.raw_writer = raw_writer
            
        if processed_writer==None:
            self.processed_writer = cv2.VideoWriter('output.avi', codec, 20.0, (640, 480))
        else:
            self.processed_writer = processed_writer
        

        #creating mediapipe objects for finding body position
        self.pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils 
        self.mp_styles = mp.solutions.drawing_styles
        self.fps = self.vid.get(cv2.CAP_PROP_FPS)
        self.raw_frames = []
        self.raw_frame = raw_frame
        self.processed_frames = []
        self.process_frame = None

        #getting deep neural network data for processing
        self.initialize_yolo()
        
    
    def __enter__(self):
        return self
  
    def __exit__(self, exc_type, exc_value, tb):
        del self
        if exc_type is not None:
            traceback.print_exception(exc_type, exc_value, tb)
            # return False # uncomment to pass exception through
        return True
    
    
    def initialize_yolo(self):
        yolo_dir = os.path.abspath("./yolo")

        # load the labels, weights, and config for the yolo model
        weights_path = os.path.sep.join([yolo_dir, "yolov3.weights"])
        config_path = os.path.sep.join([yolo_dir, "yolov3.cfg"])

        # load the labels (as list), and model
        self.yolo = cv2.dnn.readNetFromDarknet(config_path, weights_path)

        # get the output layers
        self.layer_names = self.yolo.getLayerNames()
        self.layer_names = [self.layer_names[i - 1] for i in self.yolo.getUnconnectedOutLayers()]
        
    def find_ball(self, confidence_threshold = 0.3):
        frame = self.raw_frame
        #method for detecting ball on image
        #frame = imutils.resize(frame, width=500)
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        self.yolo.setInput(blob)
        layerOutputs = self.yolo.forward(self.layer_names)
        boxes = []
        confidences = []
        classIDs = []
        for output in layerOutputs:
            # loop over each of the detections
            for detection in output:
                # get the class id and confidence for the object
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                if confidence > confidence_threshold and classID == 32:
                    # get the bounding box for the object
                    (H, W) = frame.shape[:2]
                    box = detection[0:4] * np.array([W, H, W, H])
                    (x, y, width, height) = box.astype("int")
                    # update our list of bounding box coordinates, confidences, and class IDs
                    boxes.append([x, y, width, height])
                    confidences.append(float(confidence))
                    classIDs.append(classID)
        if len(confidences) > 0:
            (x, y, w, h) = boxes[np.argmax(confidences)]
            self.ball = {
                'x':x,
                'y':y,
                'w':W,
                'h':h
            }
        else:
            self.ball = {
                'x':0,
                'y':0,
                'w':0,
                'h':0
            }
        return self.ball
    
    def get_body_vectors(self):
        
        frame = self.raw_frame
        pose = self.pose.Pose(static_image_mode=True, min_detection_confidence=0.5, model_complexity=2)
        '''
        with self.pose.Pose(
            static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as pose:
            # Convert the BGR image to RGB and process it with MediaPipe Pose.
        '''
        self.pose_landmarks = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).pose_landmarks
        #return self.pose_landmarks
                 
    def draw_on_image(self, ball=None, results=None):
        frame = self.raw_frame
        if results == None:
            results = self.pose_landmarks
        if ball == None:
            ball = self.ball
        annotated_frame = frame.copy()
        cv2.circle(annotated_frame, 
                   center = (self.ball['x'], self.ball['y']), 
                   radius=int(self.ball['h']/2), color=(0,255,0), 
                   thickness=3)
        if not self.pose_landmarks:
            pass
        else:
            self.mp_drawing.draw_landmarks(
                annotated_frame,
                self.pose_landmarks,
                self.pose.POSE_CONNECTIONS,
                landmark_drawing_spec=self.mp_styles.get_default_pose_landmarks_style())
                #annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
        self.processed_frame = annotated_frame
        del annotated_frame
    
    def save_frame(self):
        self.raw_frames.append(self.frame.copy())
        
    def process_frames(self, raw_frames=None):
        if raw_frames == None:
            raw_frames = self.raw_frames
        for frame in self.raw_frames:
            self.frame = frame
            #find the coordinates and size of the ball
            self.find_ball()
            #find the person in the image and define the location of limbs as vectors
            self.get_body_vectors()
            #draw circle on ball and lines for person limbs
            self.draw_on_image()
    
    @contextmanager
    def process_frame(self, save=False):
        try:
            #find the coordinates and size of the ball
            self.find_ball()
            #find the person in the image and define the location of limbs as vectors
            self.get_body_vectors()
            #draw circle on ball and lines for person limbs
            self.draw_on_image()
            if save == True:
                self.write_processed_frame()
            yield self.processed_frame
        finally:
            pass

            
            
    def play_video(self):
        for frame in self.processed_frames:
            cv2.imshow('frame', frame)
            cv2.waitKey(int(1000/self.fps))
            
    def write_raw_video(self):
        for frame in self.raw_frames:
            self.raw_writer.write(frame)
        
    def write_processed_frame(self):
        self.processed_writer.write(self.processed_frame)    
        
    def write_processed_video(self):
        for frame in self.processed_frames:
            self.processed_writer.write(frame)
            
    def destroy_assets(self):
        del self.pose_landmarks
        del self.ball    

In [94]:
vid = cv2.VideoCapture(0)
codec = cv2.VideoWriter_fourcc(*'XVID')
raw_writer = cv2.VideoWriter('clips/raw/raw_%02d.jpg', 0, 0, (640, 480))
processed_writer = cv2.VideoWriter('clips/processed/processed_%02d.jpg', 0, 0, (640, 480))
proc = BST_Image_Proc(vid=vid, codec=codec, raw_writer=raw_writer, processed_writer=processed_writer)
quit = False
loop_counter = 0
while True:
    loop_counter += 1
    #read from webcam
    ret, frame = proc.vid.read()
    
    #check to make sure there is an image
    if ret:
        #set the class attribute as the image from the webcam
        proc.frame = frame
        #save the frame for processing
        proc.save_frame()       
        #display image
        cv2.imshow('frame', proc.frame)
        cv2.waitKey(int(1000/proc.fps))
    
    if loop_counter > 100:
        break
vid.release()
cv2.destroyAllWindows()

In [112]:
vid = cv2.VideoCapture(0)
codec = cv2.VideoWriter_fourcc(*'XVID')
#raw_writer = cv2.VideoWriter('clips/raw/raw_%02d.jpg', 0, 0, (640, 480))
processed_writer = cv2.VideoWriter('clips/processed/processed_%02d.jpg', 0, 0, (640, 480))

file_list = os.listdir(os.getcwd() + '/clips/raw')
for image_file in file_list:
    if image_file == 'raw_%02d.jpg':
        continue
    path = os.getcwd() + "/clips/raw/" + image_file
    frame = cv2.imread(path)
    proc = BST_Image_Proc(raw_frame=frame, processed_writer=processed_writer)
    with proc.process_frame(save=True) as proc:
        print(proc)
        #proc.process_frame(save=True)
        
        

TypeError: 'NoneType' object is not callable

In [7]:
vid = cv2.VideoCapture(0)
codec = cv2.VideoWriter_fourcc(*'XVID')
raw_writer = cv2.VideoWriter('clips/raw/raw_%02d.jpg', 0, 0, (640, 480))
processed_writer = cv2.VideoWriter('clips/processed/processed_%02d.jpg', 0, 0, (640, 480))
proc = BST_Image_Proc(vid=vid, codec=codec, raw_writer=raw_writer, processed_writer=processed_writer)
quit = False
loop_counter = 0
while True:
    loop_counter += 1
    #read from webcam
    ret, frame = proc.vid.read()
    
    #check to make sure there is an image
    if ret:
        #set the class attribute as the image from the webcam
        proc.frame = frame
        #save the frame for processing
        proc.save_frame()       
        #display image
        cv2.imshow('frame', proc.frame)
        cv2.waitKey(int(1000/proc.fps))
    
    if loop_counter > 100:
        break
vid.release()
cv2.destroyAllWindows()
proc.write_raw_video()

#process recorded frames to find vectors and ball      
proc.process_frames(save=True)

#display the video with the frames annotated
proc.play_video()
  
# After the loop release the cap object
#vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

proc.write_raw_video()
proc.write_processed_video()